<a href="https://colab.research.google.com/github/vaskot1986/03MAIR--Algoritmos-de-Optimizacion--2020/blob/master/arnau_vazquez_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Nombre:** Arnau Vazquez Giner
**Url Colab:** https://colab.research.google.com/drive/1Kkwd9YZ1y2Xbh2g1-HpEOrjpGxOkk6mO?usp=sharing
**Url Github:** 

In [ ]:
pip install --upgrade pip

In [ ]:
!pip install requests
!pip install tsplib95

In [2]:
import urllib.request

In [3]:
file = 'swiss42.tsp'
urllib.request.urlretrieve('http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp',file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7fb4e3e3d400>)

In [11]:
import tsplib95
import random
from math import e,inf,exp
import copy

In [12]:
problem = tsplib95.load(file)

Nodos = list(problem.get_nodes())

Aristas = list(problem.get_edges())

In [13]:
#Se genera una solucion aleatoria con comienzo en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b,problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1], problem)
  return distancia_total + distancia(solucion[len(solucion)-1], solucion[0], problem)

In [14]:
#Busqueda aleatoria
def busqueda_aleatoria(problem, N):
  mejor_solucion = []
  mejor_distancia = inf

  for i in range(N):
    solucion = crear_solucion(Nodos)
    distancia = distancia_total(solucion, problem)

    if distancia < mejor_distancia:
      mejor_solucion = solucion
      mejor_distancia = distancia
  return mejor_solucion

s=busqueda_aleatoria(problem, 10000)

print(distancia_total(s,problem),s)

3677 [0, 9, 2, 28, 24, 40, 6, 15, 37, 7, 19, 11, 39, 25, 29, 23, 3, 1, 14, 36, 4, 12, 16, 13, 8, 10, 33, 30, 27, 17, 5, 35, 31, 20, 21, 22, 38, 34, 32, 41, 18, 26]


In [15]:
def genera_vecina(solucion, problem):
  mejor_solucion = []
  mejor_distancia = inf
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      #intercambiamos i y j usando el auxiliar
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
      distancia_vecina = distancia_total(vecina, problem)

      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion

s1=crear_solucion(Nodos)
s2=genera_vecina(s1, problem)
print(distancia_total(s1,problem),s1)
print(distancia_total(s2,problem),s2)

5104 [0, 40, 13, 4, 27, 35, 28, 14, 38, 3, 37, 5, 29, 15, 21, 10, 24, 6, 11, 12, 32, 1, 2, 8, 33, 17, 18, 31, 41, 20, 7, 36, 30, 16, 34, 25, 19, 22, 9, 39, 23, 26]
4626 [0, 15, 13, 4, 27, 35, 28, 14, 38, 3, 37, 5, 29, 40, 21, 10, 24, 6, 11, 12, 32, 1, 2, 8, 33, 17, 18, 31, 41, 20, 7, 36, 30, 16, 34, 25, 19, 22, 9, 39, 23, 26]


In [16]:
#busqueda local
def busqueda_local(problem):
  mejor_solucion = []

  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = inf

  iteracion = 0
  while(1):
    iteracion += 1
    vecina = genera_vecina(solucion_referencia, problem)
    distancia_vecina = distancia_total(vecina, problem)

    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
    else:
      print("En la iteracion", iteracion, " la mejor solucion encontrada es:",mejor_solucion)
      print("Distancia  :", mejor_distancia)
      return mejor_solucion

    solucion_referencia = vecina

busqueda_local(problem)

En la iteracion 42  la mejor solucion encontrada es: [0, 37, 15, 16, 14, 19, 13, 11, 12, 18, 20, 33, 34, 32, 3, 1, 7, 17, 36, 35, 31, 9, 23, 40, 24, 21, 39, 22, 38, 30, 29, 8, 41, 25, 10, 28, 27, 2, 4, 26, 5, 6]
Distancia  : 1745


[0,
 37,
 15,
 16,
 14,
 19,
 13,
 11,
 12,
 18,
 20,
 33,
 34,
 32,
 3,
 1,
 7,
 17,
 36,
 35,
 31,
 9,
 23,
 40,
 24,
 21,
 39,
 22,
 38,
 30,
 29,
 8,
 41,
 25,
 10,
 28,
 27,
 2,
 4,
 26,
 5,
 6]

In [17]:
#genera vecina aleatoria
def genera_vecina_aleatoria(solucion):
  i,j = sorted(random.sample( range(1,len(solucion)) , 2))
  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

def probabilidad(T,d):
  if random.random() <= exp(-1*d/T):
    return True
  else:
    return False

def bajar_temperatura(T):
  return T*.9999

def simulated_annealing(problem, TEMPERATURA=1000):
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)

  mejor_solucion = []
  mejor_distancia = inf

  N=0
  while TEMPERATURA > 0.0001:
    N+=1
    #Genera solucion vecina
    vecina = genera_vecina_aleatoria(solucion_referencia)

    #Calcula su valor de distancia
    distancia_vecina = distancia_total(vecina, problem)

    #Si es la mejor solucion de todas se guarda
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina

    #Si es la nueva vecina se cambia 
    #Si es peor se cambia segun probabilidad
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia-distancia_vecina)):
      solucion_referencia = copy.deepcopy(vecina)
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solucion encontrada es ", end="")
  print(mejor_solucion)
  print("con una distancia total de ", end="")
  print(mejor_distancia)
  return(mejor_solucion)

sol=simulated_annealing(problem,10000)

La mejor solucion encontrada es [0, 30, 29, 8, 9, 39, 22, 38, 33, 35, 36, 16, 15, 37, 17, 31, 20, 34, 32, 1, 6, 26, 18, 10, 21, 24, 40, 23, 41, 25, 11, 12, 28, 27, 2, 3, 4, 5, 13, 19, 14, 7]
con una distancia total de 1710


In [18]:
def Add_Nodo(problem, H ,T ) :
  #Mejora:Establecer una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(   list(set(range(1,len(Nodos))) - set(H) )  )


def Incrementa_Feromona(problem, T, H ) :
  #Incrementa segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T ):
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  #Mejora:Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T

In [19]:
#Colonia de hormigas
def hormigas(problem, N) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
    
  #Nodos
  Nodos = list(problem.get_nodes())
  #Aristas
  Aristas = list(problem.get_edges()) 
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  #Mejora: inicializar con valores diferentes dependiendo diferentes criterios
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]
  
  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual
  
  
  print(mejor_solucion)
  print(mejor_distancia)
  
  
hormigas(problem, 1000)


[0, 16, 17, 20, 1, 14, 31, 35, 10, 22, 27, 37, 32, 25, 4, 26, 5, 11, 41, 40, 39, 34, 2, 38, 24, 21, 9, 8, 12, 6, 3, 19, 29, 28, 33, 7, 30, 36, 13, 23, 15, 18]
3870


Algoritmo genético

In [20]:
##Algoritmo genético
#Genera una poblacion inicial de soluciones de tamaño N
def generar_poblacion(Nodos,N):
  return [crear_solucion(Nodos) for _ in range(N) ]


#Evalua la población y devuelve el mejor individuo
def Evaluar_Poblacion(poblacion, problem):
  mejor_solucion = []
  mejor_distancia = 10e100
  for p in poblacion:
    #print("solucion:", p)
    distancia_referencia = distancia_total(p, problem)
    if distancia_referencia < mejor_distancia:
      mejor_solucion = p
      mejor_distancia = distancia_referencia
  return mejor_solucion, mejor_distancia   

#Funcion de cruce. Recibe una poblacion(lista de soluciones) y devuelve la población ampliada con los hijos.
# Todos los individuos de la población son selecionados para el cruce(si la población es par)
# Podría aplicarse un proceso previo de selección para elegir los individuos que se desea cruzar.
def Cruzar(poblacion,mutacion, problem):
  #Definimos en una variable la copia de la población para ir eliminando los padres seleccionados
  poblacion_copia = copy.deepcopy(poblacion)

  #Definimos en una variable la copia de la población para ir añadiendo los hijos creados
  poblacion_final = copy.deepcopy(poblacion)

  while len(poblacion_copia) > 1:  #Iteramos mientras haya padres disponibles
    #Seleccionamos dos padres
    padre1,padre2 = random.sample(poblacion_copia   ,2)
    poblacion_copia.remove(padre1)
    poblacion_copia.remove(padre2)
    poblacion_final.extend(Descendencia([padre1,padre2],problem,mutacion))
  return poblacion_final

#Funcion para generar hijos a partir de 2 padres:
# Se elige el metodo de 1-punto de corte pero es posible usar otros n-puntos, uniforme, dependiendo del problema
def Descendencia(padres, problem,mutacion):
  #Se elige un punto de corte aleatorio:
  pc = random.sample( range(1,len(padres[0])),1)[0]
  hijo1 =  Factibilizar(padres[0][:pc] + padres[1][pc:] ,problem) 
  hijo2 =  Factibilizar(padres[1][:pc] + padres[0][pc:] ,problem)
  return [hijo1,hijo2,Mutar(hijo1,mutacion),Mutar(hijo2,mutacion)]

#Para el operador de cruce 1-punto los hijos generados no son soluciones(algunos nodos se repiten y otros no están)
def Factibilizar(solucion,problem):
  Nodos = list(problem.get_nodes())
  nodos_desaparecidos = list(set(Nodos) - set(solucion))
  #Recorremos todos los nodos, cuando haya uno que ya esté en la lista los cambiamos por uno de la lista de nodos_desaparecidos
  for i in range(len(solucion) ):
    if solucion[i] in solucion[:i]:
      #print("\tSe repite el",solucion[i] )
      #print("\tSe cambia en la posicion ",i , " " ,solucion[i] , " por ", nodos_desaparecidos[0] )
      solucion[i] = nodos_desaparecidos.pop(0) #Cambiamos el nodo y a la vez eliminamos el nodo usado de nodos_desaparecidos
  return solucion 

#Funcion de mutación. Se eligen dos nodos y se intercambia. Se podrian añadir otros operaradores
# Se hace mutaciones mutacion% de las veces
def Mutar(solucion,mutacion):
  if random.random() < mutacion:
    sel1,sel2 = sorted(random.sample(set(solucion)-{solucion[0]},2))
    return solucion[:sel1] + [solucion[sel2]] + solucion[sel1+1:sel2] + [solucion[sel1]]  + solucion[sel2+1:] 
  else:
    return solucion[::] 

#Funcion de seleccion de la población. Recibe como parametro una poblacion y 
# devuelve una poblacion a la que se ha eliminado individuos poco aptos(fitness alto) y para mantener una poblacion estable de N individuos
#Se tiene en cuenta el porcentaje elitismo pasado como parametro 
# Para los individuos que no son de la elite podríamos usar una selección de ruleta(proporcional a su fitness)
def Seleccionar(problem,poblacion, N, elitismo):
  #Se ordena la población según el fitness(tamaño del recorrido) en una lista de elementos [distancia, solucion]
  poblacion_ordenada = sorted([ [distancia_total(solucion, problem), solucion] for solucion in poblacion ], key= lambda x:x[0] )

  #Devolvemos elitismo% y el resto se eligen aleatoriamente
  return [x[1] for x in poblacion_ordenada][:int(N*elitismo)]  +   random.sample([x[1] for x in poblacion_ordenada][int(N*elitismo):] , int(N*(1-elitismo))  ) 
  

In [21]:
#Funcion principal del algoritmo genetico
#######################################################3
def algoritmo_genetico(problem=problem,N=100,mutacion=.15,elitismo=.1,generaciones=100):
  # problem = datos del problema
  # N = Tamaño de la población
  # mutacion = probabilidad de una mutación
  # elitismo = porcion de la mejor poblacion a mantener
  # generaciones = nº de generaciones a generar para finalizar

  #Genera la poblacion inicial
  Nodos = list(problem.get_nodes())
  poblacion = generar_poblacion(Nodos,N)
  
  #Inicializamos valores para la mejor solucion
  (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)

  
  #Condicion de parada
  parar = False
  n=0
  #Inciamos el cliclo de generaciones
  while(parar == False) :
    
    #Cruce de la poblacion(incluye mutación)
    poblacion = Cruzar(poblacion, mutacion, problem)
  
    #Seleccionamos la población
    poblacion = Seleccionar(problem,poblacion, N, elitismo)
      
    #Evaluamos la nueva población
    (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)
  
    print("Generacion #", n, "\nLa mejor solución es:" , mejor_solucion, "\ncon distancia " , mejor_distancia, "\n")
 
    #Numero de generaciones. Criterio de parada
    if n==generaciones:
      parar = True
    n +=1
    
  return mejor_solucion
  
#1º intento  :2113
#sol = algoritmo_genetico(problem=problem,N=100,mutacion=.15,elitismo=.1,generaciones=200)

#2º intento. Aumentamos la poblacion:1654
#sol = algoritmo_genetico(problem=problem,N=500,mutacion=.15,elitismo=.1,generaciones=200)

#3º intento. Aumentamos las generaciones:2055
#sol = algoritmo_genetico(problem=problem,N=500,mutacion=.15,elitismo=.1,generaciones=250)

#4º intento. Aumentamos el elitismo: 1728
#sol = algoritmo_genetico(problem=problem,N=500,mutacion=.15,elitismo=.40,generaciones=250)

#5º intento. Aumentamos la mutacion : 1653
sol = algoritmo_genetico(problem=problem,N=500,mutacion=.3,elitismo=.40,generaciones=250)

Generacion # 0 
La mejor solución es: [0, 16, 36, 35, 34, 20, 17, 8, 18, 31, 7, 15, 29, 14, 33, 19, 37, 28, 30, 6, 25, 41, 4, 22, 21, 38, 32, 26, 27, 2, 1, 5, 13, 39, 40, 9, 3, 11, 10, 12, 23, 24] 
con distancia  3620 

Generacion # 1 
La mejor solución es: [0, 16, 36, 35, 34, 20, 17, 8, 18, 31, 7, 15, 29, 14, 33, 19, 37, 28, 30, 6, 25, 41, 4, 22, 21, 38, 32, 26, 27, 2, 1, 5, 13, 39, 40, 9, 3, 11, 10, 12, 23, 24] 
con distancia  3620 

Generacion # 2 
La mejor solución es: [0, 20, 41, 23, 19, 32, 30, 3, 38, 11, 25, 4, 5, 31, 17, 36, 7, 15, 1, 2, 16, 14, 27, 26, 29, 40, 28, 34, 33, 35, 8, 39, 9, 10, 12, 18, 6, 21, 22, 24, 13, 37] 
con distancia  3571 

Generacion # 3 
La mejor solución es: [0, 20, 41, 23, 19, 32, 30, 3, 38, 11, 25, 4, 5, 31, 17, 36, 7, 15, 1, 2, 16, 14, 27, 26, 29, 40, 28, 34, 33, 35, 8, 39, 9, 10, 12, 18, 6, 21, 22, 24, 13, 37] 
con distancia  3571 

Generacion # 4 
La mejor solución es: [0, 16, 36, 35, 34, 20, 17, 8, 18, 31, 7, 15, 29, 14, 33, 19, 13, 2, 32, 3, 4, 25,